In [1]:
from ImgProcessor.convertDcm2Jpg import convertDcm2Jpg

dcmpath = "patient-data/p001.dcm"
targetpath = 'patient-data/p001.jpg'

convertDcm2Jpg(dcmpath, targetpath)

In [8]:
from ImgProcessor.readPatientData import readPatientData

csvpath = "patient-data/p001.csv"

patient001 = readPatientData(csvpath)

#print(patient001.at[0, 'PatientName'])

john doe


In [2]:
!python PredictionModel/run-pred-model.py patient-data/p001.jpg

{'Atelectasis': 0.21017581,
 'Cardiomegaly': 0.056692116,
 'Pleural_Thickening': 0.5006808,
 'Pneumonia': 0.2616243}


In [2]:
import csv
import random
import os

# Set the number of samples you want in the CSV file
num_samples = 1

# Specify the path for the CSV file
csv_file_path = 'patient-data/p001.csv'

# Define CSV header
csv_header = ['PatientID', 'PatientName', 'PatientBirthDate', 'PatientSex', 'StudyDate', 'Modality', 'BodyPartExamined']

# Generate and write sample patient data to CSV
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Write the header
    csv_writer.writerow(csv_header)

    # Generate and write sample data
    for _ in range(num_samples):
        patient_id = '001'
        patient_name = 'john doe'
        patient_birthdate = '01-01-1990'
        patient_sex = random.choice(['M', 'F'])
        study_date = '01-01-2024'
        modality = 'CR'  # CR stands for computed radiography (used for X-ray)
        body_part_examined = 'CHEST'

        # Write the data to the CSV file
        csv_writer.writerow([patient_id, patient_name, patient_birthdate, patient_sex, study_date, modality, body_part_examined])

print(f"Sample patient data has been saved to: {csv_file_path}")


Sample patient data has been saved to: patient-data/p001.csv
